# Full & Reduced model with protocols from Hendrickson (2011)

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ~/workspace/bgcellmodels/GilliesWillshaw

In [ ]:
# Standard library imports
import re, functools

# Import all our functions
from evalmodel.stn_model_evaluation import *
from evalmodel import proto_common, proto_hendrickson

# Full STN model

Build the Gillies & Willshaw STN model

In [ ]:
full_model = StnModel.Gillies2005
red_model = StnModel.Gillies_FoldBush_Tapered

# Make cell model and evaluator
evaluator = StnModelEvaluator(full_model, PhysioState.NORMAL)

# Register protocol functions
init_funcs = [proto_hendrickson.init_sim_impl]
setup_funcs = [proto_hendrickson.make_all_synapses]

rec_funcs = [
    proto_common.rec_spikes,
    proto_common.rec_Vm
]

plot_funcs = [
    functools.partial(proto_common.plot_all_spikes,
                      trace_filter=lambda trace: re.search('AP_'+Pop.CTX.name.lower(), trace),
                      color='g', title='NMDA spikes'),
    functools.partial(proto_common.plot_all_spikes,
                      trace_filter=lambda trace: re.search('AP_'+Pop.STR.name.lower(), trace),
                      color='b', title='AMPA spikes'),
    proto_common.plot_all_Vm,
]

setup_kwargs = {
    'tstop': 1000.0,
    'num_syn_ampa': 50,
    'num_syn_nmda': 50,
    'num_syn_gabaa': 0,
    'rec_pre_pop_spikes': ['str', 'ctx', 'stn'],
}

setup_kwargs_getters = {
    'rec_hoc_objects': lambda kwargs: {
                                'soma': kwargs['nrnsim'].SThcell[0].soma(0.5),
                                'dend': kwargs['nrnsim'].SThcell[0].dend1[7](0.8)
    }
}

evaluator.register_keyword_funcs(
            proto_init_funcs = init_funcs,
            proto_setup_funcs_pre = setup_funcs,
            proto_rec_funcs = rec_funcs,
            proto_plot_funcs = plot_funcs,
            proto_setup_kwargs_const = setup_kwargs,
            proto_setup_kwargs_getters = setup_kwargs_getters)

evaluator.setup_keyword_protocol(pre_model=full_model)

In [ ]:
# inspect the model in NEURON GUI (this pops up a GUI window outside browser)
# click __Tools > ModelView > Soma > Point Processes > GABAsyn & GLUsyn
# from neuron import gui

# import matplotlib.pyplot as plt
# import bgcellmodels.morphology.morph_gui as morphology
# from mpl_toolkits.mplot3d import Axes3D

# plt.figure()
# ax = plt.subplot(111, projection='3d')
# ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# # Plot shape of our cell
# h.define_shape()
# morphology.shapeplot(h, ax)

# # Mark locations of inputs
# all_inputs = evaluator._iproto_data['stim_data']
# marker_colors = 'rgby'
# for i, syn in enumerate(all_inputs['synapses']):
#     morphology.mark_locations(h, 
#                             syn.get_segment().sec,
#                             syn.get_loc())

## Run full STN model

Run the protocol

In [ ]:
# Run protocol
evaluator.run_keyword_protocol(plot_traces=True)

# Reduced STN model

Collapse dendritic trees using BranchZip algorithm, then remap synapses

In [ ]:
# # red_model = StnModel.Gillies_FoldBush_Tapered
# red_model = StnModel.Gillies_FoldMarasco_Legacy

# # Model reduction
# evaluator.build_cell(red_model)
# evaluator.target_model = red_model

## Plot morphology & synapses

Run the same stimulation protocol with the reduced model.

In [ ]:
# # Run Protocol
# evaluator._setup_proto(proto)

# # import matplotlib.pyplot as plt
# # import bgcellmodels.morphology.morph_gui as morphology
# # from mpl_toolkits.mplot3d import Axes3D

# plt.figure()
# ax = plt.subplot(111, projection='3d')
# ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# # Plot shape of our cell
# h.define_shape()
# morphology.shapeplot(h, ax)

# # Mark locations of inputs
# all_inputs = evaluator.get_all_inputs(red_model)
# all_synapses = all_inputs['synapses']
# print("Mapped {} synapses".format(len(all_synapses)))

# marker_colors = 'rgby'
# for i, syn in enumerate(all_synapses):
#     morphology.mark_locations(h, 
#                             syn.get_segment().sec,
#                             syn.get_loc())

## Run reduced model

In [ ]:
# evaluator._run_proto(proto)
# evaluator.plot_traces(proto)